In [2]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [3]:
df = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF gtpase code\RF_molecules_Newfeatures_GTPase_KRas.csv")
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\TheSh\AppData\Local\Temp\ipykernel_11336\2236944027.py:1: SyntaxWarning: invalid escape sequence '\T'
  df = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF gtpase code\RF_molecules_Newfeatures_GTPase_KRas.csv")


,ChEMBL ID,Frequency,Avg Predicted Value
0,CHEMBL4594350,1.0,8.106427
1,CHEMBL2359966,1.0,7.497036
2,CHEMBL2361370,1.0,7.419709
3,CHEMBL3989958,1.0,7.372708
4,CHEMBL3707183,1.0,7.497036


In [6]:
list = []
for i in df["ChEMBL ID"]:
    molecule = new_client.molecule
    m1 = molecule.filter(chembl_id=i).only(
        ['molecule_chembl_id', 'pref_name', 'molecule_structures'])
    targets = pd.DataFrame.from_dict(m1)
    #print(targets['molecule_chembl_id'][0])
    if targets['molecule_chembl_id'][0] == i:
        list.append([i,targets['pref_name'][0]])
        print(i,targets['pref_name'][0])
finals_2 = pd.DataFrame(list)

CHEMBL4594350 ADAGRASIB
CHEMBL2359966 QUININE SULFATE
CHEMBL2361370 PENBUTOLOL SULFATE
CHEMBL3989958 IVOSIDENIB
CHEMBL3707183 QUINIDINE SULFATE
CHEMBL4297091 CODEINE SULFATE
CHEMBL3187985 APOMORPHINE HYDROCHLORIDE
CHEMBL2103744 MORPHINE SULFATE
CHEMBL3183409 APALUTAMIDE
CHEMBL4535757 SOTORASIB


In [25]:
finals_2

,0,1
0,CHEMBL1200953,HYDROCORTISONE PROBUTATE
1,CHEMBL717,MEDROXYPROGESTERONE ACETATE
2,CHEMBL2107389,GESTONORONE CAPROATE
3,CHEMBL3707377,SEGESTERONE ACETATE
4,CHEMBL1411731,TALAMPICILLIN HYDROCHLORIDE
5,CHEMBL1201101,TESTOSTERONE CYPIONATE
6,CHEMBL1201064,FLUOROMETHOLONE ACETATE
7,CHEMBL1200975,CLOCORTOLONE PIVALATE
8,CHEMBL1201749,DIFLUPREDNATE
9,CHEMBL1200545,DIFLORASONE DIACETATE
